# Data Import

In [1]:
# Import libraries
import os
import math
import pandas as pd

In [2]:
date = "08-27-2025"
data_dict = {}

In [3]:
graph_str       = "../graphs/" + date + "/"
analysis_str    = "../analysis/" + date + "/"
directory_str   = "../data/" + date + "/"

for dir in os.listdir(directory_str):
    data_dict[dir] = []

    tmp_graph_str     = graph_str + dir + "/"
    tmp_directory_str = directory_str + dir + "/"

    if os.path.isdir(analysis_str) == False:
        os.makedirs(analysis_str)

    if os.path.isdir(tmp_graph_str) == False:
        os.makedirs(tmp_graph_str)

    for file in os.listdir(tmp_directory_str):
        data_dict[dir].append(pd.read_csv(tmp_directory_str + file, delimiter=";"))

# Data Cleaning

In [4]:
COL_NAMES = {
    "X_COL_1": "Flow EZ #2 (12807)",
    "Y_COL_1": "Flow Unit #1 [Flow EZ #2 (12807)]",
}
num_graphs = math.floor(COL_NAMES.keys().__len__() / 2)
i_range = list(range(1, num_graphs + 1))

filtered_dict = {}

COL_LIST = list(COL_NAMES.values())
COL_LIST.append("Time")

In [5]:
for key in data_dict.keys():
    filtered_dict[key] = []

    for df in data_dict[key]:
        filtered_dict[key].append(df[COL_LIST])

# Functions

In [6]:
import numpy as np
import scipy.stats as stats

from typing import Tuple
from scipy.optimize import curve_fit

In [7]:
def find_logarithmic_fit(df: pd.DataFrame) -> list[str]:
    """
    Use scipy to find the logarithmic fit and fitting parameters
    """
    log_eqs = []

    for i in i_range:
        x_col = COL_NAMES[f"X_COL_{i}"]
        y_col = COL_NAMES[f"Y_COL_{i}"]
        log_fit = f"log_fit_{i}"

        log_x = np.log(df[x_col])
        y = df[y_col]

        a_log, b_log, r_log, _, _ = stats.linregress(log_x, y)

        df[log_fit] = a_log * log_x + b_log

        y_true     = df[y_col]
        y_pred_log = df[log_fit]
        ss_res_log = np.sum((y_true - y_pred_log) ** 2)
        ss_tot_log = np.sum((y_true - np.mean(y_true)) ** 2)
        r2_log = 1 - (ss_res_log / ss_tot_log)

        if not np.isnan(a_log):
            log_eqs.append((
                f"Log: y = {a_log:.3f} ln(x) + {b_log:.3f}\n"
                f"R² = {r2_log:.3f}"
            ))
        else:
            df['log_fit'] = np.nan
            log_eqs.append("Logarithmic fit failed")

    return log_eqs

In [8]:
def find_logistic_fit(df: pd.DataFrame) -> list[str]:
    """
    Use scipy to find the logistic fit and fitting parameters
    """
    logistic_eqs = []

    # Logistic function definition
    def logistic(x, L, k, x0):
        return L / (1 + np.exp(-k * (x - x0)))

    for i in i_range:
        x_col = COL_NAMES[f"X_COL_{i}"]
        y_col = COL_NAMES[f"Y_COL_{i}"]
        logistic_fit = f"logistic_fit_{i}"

        # Initial parameter guess: [max y, slope, midpoint]
        initial_guess = [df[y_col].max(), 1, df[x_col].median()]

        # Fit the logistic model
        try:
            params, _ = curve_fit(logistic, df[x_col], df[y_col], p0=initial_guess, maxfev=10000)
            L, k, x0_log = params
            df[logistic_fit] = logistic(df[x_col], L, k, x0_log)

            # Calculate R²
            y_true = df[y_col]
            y_pred = df[logistic_fit]
            ss_res = np.sum((y_true - y_pred) ** 2)
            ss_tot = np.sum((y_true - np.mean(y_true)) ** 2)
            r2_logistic = 1 - (ss_res / ss_tot)

            logistic_eqs.append((
                f"Logistic: y = {L:.2f} / (1 + e^(-{k:.2f}(x - {x0_log:.2f})))\n"
                f"R² = {r2_logistic:.3f}"
            ))
        except RuntimeError:
            df[logistic_fit] = np.nan
            logistic_eqs.append("Logistic fit failed")

    return logistic_eqs

In [9]:
def find_lin_fit(df: pd.DataFrame) -> list[str]:
    """
    Use scipy to find the linear fit and fitting parameters
    """
    lin_eqs = []

    for i in i_range:
        x_col = COL_NAMES[f"X_COL_{i}"]
        y_col = COL_NAMES[f"Y_COL_{i}"]
        lin_fit = f"lin_fit_{i}"

        a_lin, b_lin, r_lin, _, _ = stats.linregress(df[x_col], df[y_col])

        df[lin_fit] = a_lin * df[x_col] + b_lin

        y_true     = df[y_col]
        y_pred_lin = df[lin_fit]
        ss_res_lin = np.sum((y_true - y_pred_lin) ** 2)
        ss_tot_lin = np.sum((y_true - np.mean(y_true)) ** 2)
        r2_lin = 1 - (ss_res_lin / ss_tot_lin)

        lin_eqs.append((
            f"Linear: y = {a_lin:.3f}x + {b_lin:.3f}\n"
            f"R² = {r2_lin:.3f}"
        ))

    return lin_eqs

In [10]:
def melt_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Melt data for legend support
    """
    # name_vars = [f"Logarithmic Fit {i}" for i in i_range] + [f"Linear Fit {i}" for i in i_range] + [f"Logistic Fit {i}" for i in i_range]
    # value_vars = [f"log_fit_{i}" for i in i_range] + [f"lin_fit_{i}" for i in i_range] + [f"logistic_fit_{i}" for i in i_range]
    name_vars = [f"Linear Fit {i}" for i in i_range] + [f"Logistic Fit {i}" for i in i_range]
    value_vars = [f"lin_fit_{i}" for i in i_range] + [f"logistic_fit_{i}" for i in i_range]

    df_melt = pd.melt(
        df,
        id_vars=COL_NAMES.values(),
        value_vars=value_vars,
        var_name='Fit Type',
        value_name='Fit Value'
    )

    # Rename for legend
    fit_labels = {value_vars[i]: name_vars[i] for i in range(len(name_vars))}
    df_melt['Fit Type'] = df_melt['Fit Type'].map(fit_labels)

    return df_melt

In [11]:
def chi_square_test(df_melt: pd.DataFrame, chip: str, alpha: float = 0.05):
    """
    Perform Chi-Square test
    """
    def calculate_bins(data: list) -> int:
        """
        Calculate the number of histogram bins using the Freedman-Diaconis rule.
        """
        # Pre-process
        data = np.asarray(data)
        data = data[~np.isnan(data)]

        q75, q25 = np.percentile(data, [75 ,25])
        iqr = q75 - q25
        n = len(data)

        # Fallback: just one bin
        if iqr == 0 or n <= 1:
            return 1

        bin_width = 2 * iqr / (n ** (1 / 3))

        # Avoid division by zero
        if bin_width == 0:
            return 1

        data_range = data.max() - data.min()
        bins = int(np.ceil(data_range / bin_width))

        # Always at least one bin
        return max(bins, 1)

    for i in i_range:
        x_col = COL_NAMES[f"X_COL_{i}"]
        y_col = COL_NAMES[f"Y_COL_{i}"]

        # Replace with your DataFrame and column name
        df = df_melt.copy()
        x_data = df[x_col]
        y_data = df[y_col]

        # Calculate the number of bins using Freedman-Diaconis rule
        x_bins = calculate_bins(x_data)
        y_bins = calculate_bins(y_data)

        # Bin the x and y data
        df['x_binned'] = pd.cut(x_data, bins=x_bins)
        df['y_binned'] = pd.cut(y_data, bins=y_bins)

        # Create contingency table (cross-tab of binned values)
        contingency_table = pd.crosstab(df['x_binned'], df['y_binned'])

        # Perform chi-squared test
        chi2, p, dof, expected = stats.chi2_contingency(contingency_table)

        # Write data to file
        with open(analysis_str + f"{chip}_chi_square_results_{i}.txt", "w") as f:
            # Log results to file
            f.write(f"Chi-squared statistic: {chi2}\n")
            f.write(f"Degrees of freedom: {dof}\n")
            f.write(f"Optimal bin count: x = {x_bins}, y = {y_bins}\n")
            f.write(f"Expected frequencies:\n{expected}\n")
            f.write(f"P-value: {p}\n")

            # Interpret the result
            if p < alpha:
                f.write("Result: Reject the null hypothesis — association exists between binned x and y.")
            else:
                f.write("Result: Fail to reject the null — no significant association.")

In [12]:
def anova_test(y_vals: list, alpha: float = 0.05) -> None:
    """
    Perform one-way ANOVA test
    """
    # Number of groups and total number of observations
    k = len(y_vals)
    n = 0

    for group in y_vals:
        n += len(group)

    # Perform one-way ANOVA
    f_statistic, p_value = stats.f_oneway(*y_vals)

    # Degrees of freedom
    df_between = k - 1
    df_within = n - k

    # Calculate critical F value
    f_critical = stats.f.ppf(1 - alpha, df_between, df_within)

    # Write data to file
    with open(analysis_str + "anova_results.txt", "w") as f:
        # Log results to file
        f.write(f"F-statistic: {f_statistic:.5f}\n")
        f.write(f"p-value: {p_value:.5f}\n")
        f.write(f"F-critical (alpha = {alpha}): {f_critical:.5f}\n")

        # Interpret the result
        if f_statistic > f_critical:
            f.write(f"Reject the null hypothesis: At least one group mean is different.")
        else:
            f.write(f"Fail to reject the null hypothesis: No significant difference between group means.")

# Data Analysis

In [13]:
from plotnine import (
    ggplot,
    aes,
    geom_point,
    geom_line,
    ggtitle,
    labs,
    annotate,
    xlim,
    ylim,
    scale_color_manual,
)

In [14]:
X_LABEL = "Pressure In (mb)"
Y_LABEL = "Flow Rate Out (µL/min)"

In [15]:
# Make variables
data_list = []
plot_layers = []
colors = [
    "black",
    "purple",
    "orange"
]

x_min = 0
y_min = 0
x_max = 100
y_max = 1000
annotation_step = 75

In [16]:
# Graph all the data
for key in filtered_dict.keys():
    df = pd.DataFrame()
    chip_list = filtered_dict[key]

    for i, df in enumerate(chip_list):
        # Find applicable fits
        # log_eqs      = find_logarithmic_fit(df)
        lin_eqs      = find_lin_fit(df)
        logistic_eqs = find_logistic_fit(df)

        # Melt the data
        df_melt = melt_data(df)

        for j in i_range:
            x_col = COL_NAMES[f"X_COL_{j}"]
            y_col = COL_NAMES[f"Y_COL_{j}"]

            x_min = 0
            y_min = df_melt[y_col].min()
            x_max = df_melt[x_col].max()
            y_max = df_melt[y_col].max()

            annotation_step = ((y_max - y_min) / 4) / 3

            # Make the plot
            plot =  (
                ggplot(df_melt, aes(x_col, y_col)) +
                geom_point() +
                ggtitle(f"{Y_LABEL} vs {X_LABEL}") +
                geom_line(
                    aes(
                        y='Fit Value',
                        color='Fit Type'
                    ),
                    data=df_melt[df_melt["Fit Type"].str.endswith(f"{j}")]
                ) +
                scale_color_manual(
                    values = {
                        # f'Logarithmic Fit {j}': 'blue',
                        f'Linear Fit {j}': 'red',
                        f'Logistic Fit {j}': 'green'
                    },
                    labels={
                        # f'Logarithmic Fit {j}': 'Logarithmic Fit',
                        f'Linear Fit {j}': 'Linear Fit',
                        f'Logistic Fit {j}': 'Logistic Fit'
                    }
                ) +
                labs(
                    x=X_LABEL,
                    y=Y_LABEL,
                    color="Model Fit"
                ) +
                xlim(0, x_max) +
                ylim(0, y_max) +
                annotate(
                    "text",
                    x=0,
                    y=y_max - annotation_step * 1,
                    label=logistic_eqs[j - 1],
                    ha='left',
                    va='top',
                    size=8,
                    color='green'
                ) +
                annotate(
                    "text",
                    x=0,
                    y=y_max - annotation_step * 0,
                    label=lin_eqs[j - 1],
                    ha='left',
                    va='top',
                    size=8,
                    color='red'
                )
                # ) +
                # annotate(
                #     "text",
                #     x=0,
                #     y=y_max - annotation_step * 0, # If included, this should be the zero to keep with legend ordering
                #     label=log_eqs[j - 1],
                #     ha='left',
                #     va='top',
                #     size=8,
                #     color='blue'
                # )
            )

            # Save the plot
            plot.save(graph_str + f"{key}/graph_{i}-{j}.png", width=16, height=9, dpi=600)

            # Add data to the analysis list
            data_list.append(df_melt[y_col].tolist())

        # Perform Chi-Square test
        chi_square_test(df_melt, key)

/tmp/ipykernel_404849/3353449167.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/tmp/ipykernel_404849/217409766.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/srv/vips_data_analysis/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:615: PlotnineWarning: Saving 16 x 9 in image.
/srv/vips_data_analysis/.venv/lib/python3.10/site-packages/plotnine/ggplot.py:616: PlotnineWarning: Filename: ../graphs/08-27-2025/chip_1/graph_0-1.png
/srv/vips_data_analysis/.venv/lib/python3.10/site-packages/plotnine/l

In [ ]:
# Perform anova analysis
anova_test(data_list)